In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
def to_xyz(az, el):
    x = np.cos(el)*np.cos(az)
    y = np.cos(el)*np.sin(az)
    z = np.sin(el)
    return (x,y,z)

def to_azel(x, y, z):
    el = np.arcsin(z)
    az = np.arctan2(y, x)    
    return az, el
    
def Ry(theta):
  return np.matrix([[ np.cos(theta), 0, np.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-np.sin(theta), 0, np.cos(theta)]])

def Rx(theta):
  return np.matrix([[ 1, 0           , 0           ],
                   [ 0, np.cos(theta),-np.sin(theta)],
                   [ 0, np.sin(theta), np.cos(theta)]])



def PointingError(expId):
    mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument='LSSTComCam')
    azStart = mData['AZSTART']
    azEnd = mData['AZEND']
    elStart = mData['ELSTART']
    elEnd = mData['ELEND']
    expTime = mData['EXPTIME']
    
    
    rot = 200.0 / 3600.0 * np.pi / 180.0
    rx = Rx(rot)
    (x,y,z) = to_xyz(azStart*np.pi/180.0, elStart*np.pi/180.0)
    out = rx.dot((x,y,z))
    out = np.array(out)[0]
    azStart2, elStart2 = to_azel(out[0], out[1], out[2])
    azStart2 = azStart2*180.0/np.pi
    elStart2 = elStart2*180.0/np.pi

    (x,y,z) = to_xyz(azEnd*np.pi/180.0, elEnd*np.pi/180.0)
    out = rx.dot((x,y,z))
    out = np.array(out)[0]
    azEnd2, elEnd2 = to_azel(out[0], out[1], out[2])
    azEnd2 = azEnd2*180.0/np.pi
    elEnd2 = elEnd2*180.0/np.pi
    
    azMove = azStart - azEnd
    elMove = elStart - elEnd
    azMove2 = azStart2 - azEnd2
    elMove2 = elStart2 - elEnd2
    
    azMoveDelta = (azMove - azMove2) * 3600.0
    elMoveDelta = (elMove - elMove2) * 3600.0
    
    pointingDelta = np.sqrt(elMoveDelta**2 + (azMoveDelta * np.cos(elStart * np.pi / 180.0))**2)
    print(expTime, azStart, elStart, elMoveDelta, azMoveDelta, pointingDelta)
    return [azStart, elStart, pointingDelta]


In [ ]:
instrument = 'LSSTComCam'
butler = butlerUtils.makeDefaultButler(instrument)

startDay = 20241112
endDay = 20241120

fwhms = []
pointingErrors = []
dayObss = []
azs = []
els = []
dayObs = startDay
while dayObs <= endDay:
    table = pd.read_json(f'/project/rubintv/LSSTComCam/sidecar_metadata/dayObs_{dayObs}.json').T
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure",
                where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(f"For {dayObs}, there are {len(exposureList)} exposures")
    numPoints = 0
    for [id,record] in exposureList:
        if record.observation_type not in ['science']:
            continue
        try:
            seqNum = record.seq_num
            fwhm = table.loc[seqNum]['PSF FWHM']
            fwhms.append(fwhm)
            [azStart, elStart, pointingError] = PointingError(id)
            pointingErrors.append(pointingError)
            dayObss.append(dayObs - startDay)
            azs.append(azStart)
            els.append(elStart)
            numPoints += 1
        except:
            continue

    dayObs = calcNextDay(dayObs)


In [ ]:
len(pointingErrors)

In [ ]:
scatt = plt.scatter(pointingErrors, fwhms, c=els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(scatt)


In [ ]:
scatt2 = plt.scatter(azs, pointingErrors, c=els, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(scatt2)


In [ ]:
scatt3 = plt.scatter(els, pointingErrors, c=azs, cmap=plt.cm.coolwarm)
cb4 = plt.colorbar(scatt3)


In [ ]:
rad_azs = []
for az in azs:
rad_azs = np.array(azs) * np.pi / 180.0

fig=plt.figure(figsize=(8,8))
ax1 = plt.subplot(111, projection='polar')
ax1.set_title(f"Sky coverage - ComCam campaign 20241028-20241119")
ax1.invert_yaxis()
ax1.set_rlim(90,0)
scatt5 = ax1.scatter (rad_azs, els, c = pointingErrors, cmap=plt.cm.coolwarm)
plt.colorbar(scatt5)
#plt.savefig("/home/c/cslage/u/ComCam/data/Sky_Coverage_20Nov24.png")